In [1]:
import cv2 
import os
from PIL import Image
import numpy as np

In [2]:
image_directory='dataset/'
no_tumor_images=os.listdir(image_directory+ 'no/')
yes_tumor_images=os.listdir(image_directory+ 'yes/')
dataset=[]
label=[]
INPUT_SIZE=64
      


In [3]:
# checking image is jpg or not and making dataset and label
for i , image_name in enumerate(no_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'no/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE)) 
        dataset.append(np.array(image))
        label.append(0)
for i , image_name in enumerate(yes_tumor_images):
    if(image_name.split('.')[1]=='jpg'):
        image=cv2.imread(image_directory+'yes/'+image_name)
        image=Image.fromarray(image, 'RGB')
        image=image.resize((INPUT_SIZE,INPUT_SIZE)) 
        dataset.append(np.array(image))
        label.append(1)

In [4]:
# dataset and label 
print(len(dataset))
print(len(label))

3000
3000


In [5]:
#changing data into numpy array
dataset=np.array(dataset)
label=np.array(label)

In [6]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(dataset,label,test_size=0.2,random_state=0)

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.utils import normalize


In [8]:
# normalizing data for binary classification
x_train=normalize(x_train, axis=1)
x_test=normalize(x_test, axis=1)

In [9]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense

In [10]:
# model building

#sequential model
model=Sequential()
#firstlayer: putting shape of the input
#Conv2D(filter,kersnal_size)using 32 no. of convolution and
# using (3,3) kernal size and input size of 64x64
model.add(Conv2D(32,(3,3), input_shape=(INPUT_SIZE,INPUT_SIZE,3)))
#adding activation fuction
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#initializing kernel
model.add(Conv2D(32,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,(3,3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

#flattening
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))

#output layer using 1 coz using binary classification prob 
#if we used categorical classification we would have used 2
model.add(Dense(1))
#using sigmoid function as binary cross entropy 
#if categorical cross entropy we use softmax
model.add(Activation('sigmoid'))

#compiling the model using binary_cross_entrpy
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),optimizer='adam',metrics=['accuracy'])

#fitting the model
model.fit(x_train,y_train,batch_size=16, verbose=1,epochs=10, validation_data=(x_test,y_test),shuffle=False)
#saving model
model.save('BrainTumor10Epochs.h5')




Epoch 1/10


c:\anaconda\lib\site-packages\keras\backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


150/150 [==============================] - 8s 49ms/step - loss: 0.5532 - accuracy: 0.7188 - val_loss: 0.4970 - val_accuracy: 0.7617
Epoch 2/10
150/150 [==============================] - 7s 50ms/step - loss: 0.4072 - accuracy: 0.8267 - val_loss: 0.3549 - val_accuracy: 0.8467
Epoch 3/10
150/150 [==============================] - 7s 45ms/step - loss: 0.3141 - accuracy: 0.8767 - val_loss: 0.2852 - val_accuracy: 0.8883
Epoch 4/10
150/150 [==============================] - 7s 46ms/step - loss: 0.2377 - accuracy: 0.9137 - val_loss: 0.2172 - val_accuracy: 0.9317
Epoch 5/10
150/150 [==============================] - 7s 45ms/step - loss: 0.1750 - accuracy: 0.9413 - val_loss: 0.2238 - val_accuracy: 0.9000
Epoch 6/10
150/150 [==============================] - 7s 46ms/step - loss: 0.1211 - accuracy: 0.9613 - val_loss: 0.1487 - val_accuracy: 0.9483
Epoch 7/10
150/150 [==============================] - 7s 46ms/step - loss: 0.0863 - accuracy: 0.9717 - val_loss: 0.1065 - val_accuracy: 0.9633
Epoch 8/10

In [12]:
model.evaluate(x_test,y_test)

19/19 [==============================] - 0s 14ms/step - loss: 0.1088 - accuracy: 0.9683


[0.10881008207798004, 0.9683333039283752]

In [11]:
#testing with prediction
from keras.models import load_model

model1=load_model('BrainTumor10Epochs.h5')
image=cv2.imread('C:\\Users\\Ujjwal Shandilya\\ml code\\brain tumor\\prediction\\pred5.jpg')
img=Image.fromarray(image)
img=img.resize((64,64))
img=np.array(img)
img=np.expand_dims(img,axis=0)
result=model.predict(img)
print(result[0][0])


1/1 [==============================] - 0s 82ms/step
1.0
